In [1]:
import numpy as np
import pandas as pd

import os
from datetime import datetime
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

# Palette setting
import seaborn as sns
sns.set_palette('Set2')
# 한글 폰트 깨짐 해결
import matplotlib
matplotlib.rcParams['font.family'] ='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] =False

In [2]:
# Warning 안뜨도록
import warnings
warnings.filterwarnings("ignore")

### Data path 접근하면서 dictionary 생성

In [61]:
root_path = 'c:/Users/nj/nj/Datasets/BRATS2017/'

file_dict = []

for dir in ['HGG', "LGG"]:
    path1 = os.path.join(root_path,dir)
    for dir2 in sorted(os.listdir(path1)):
        path2 = os.path.join(path1, dir2)
        if 'DS_Store' in path2:
            continue

        dict_temp = {}
        for file in sorted(os.listdir(path2)):
            for type in ['flair', 't1.', 't1ce', 't2', 'seg']:
                if type in file:
                    if type == 't1.':
                        type = 't1'
                    dict_temp[type] = os.path.join(path2, file)[34:].replace('\\','/')
        dict_temp['label'] = 0 if dir=='HGG' else 1
        file_dict.append(dict_temp)

In [74]:
pd.DataFrame(file_dict)

,flair,seg,t1,t1ce,t2,label
0,HGG/Brats17_2013_10_1/Brats17_2013_10_1_flair....,HGG/Brats17_2013_10_1/Brats17_2013_10_1_seg.ni...,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t1.nii.gz,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t1ce.n...,HGG/Brats17_2013_10_1/Brats17_2013_10_1_t2.nii.gz,0
1,HGG/Brats17_2013_11_1/Brats17_2013_11_1_flair....,HGG/Brats17_2013_11_1/Brats17_2013_11_1_seg.ni...,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t1.nii.gz,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t1ce.n...,HGG/Brats17_2013_11_1/Brats17_2013_11_1_t2.nii.gz,0
2,HGG/Brats17_2013_12_1/Brats17_2013_12_1_flair....,HGG/Brats17_2013_12_1/Brats17_2013_12_1_seg.ni...,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t1.nii.gz,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t1ce.n...,HGG/Brats17_2013_12_1/Brats17_2013_12_1_t2.nii.gz,0
3,HGG/Brats17_2013_13_1/Brats17_2013_13_1_flair....,HGG/Brats17_2013_13_1/Brats17_2013_13_1_seg.ni...,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t1.nii.gz,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t1ce.n...,HGG/Brats17_2013_13_1/Brats17_2013_13_1_t2.nii.gz,0
4,HGG/Brats17_2013_14_1/Brats17_2013_14_1_flair....,HGG/Brats17_2013_14_1/Brats17_2013_14_1_seg.ni...,HGG/Brats17_2013_14_1/Brats17_2013_14_1_t1.nii.gz,HGG/Brats17_2013_14_1/Brats17_2013_14_1_t1ce.n...,HGG/Brats17_2013_14_1/Brats17_2013_14_1_t2.nii.gz,0
...,...,...,...,...,...,...
280,LGG/Brats17_TCIA_644_1/Brats17_TCIA_644_1_flai...,LGG/Brats17_TCIA_644_1/Brats17_TCIA_644_1_seg....,LGG/Brats17_TCIA_644_1/Brats17_TCIA_644_1_t1.n...,LGG/Brats17_TCIA_644_1/Brats17_TCIA_644_1_t1ce...,LGG/Brats17_TCIA_644_1/Brats17_TCIA_644_1_t2.n...,1
281,LGG/Brats17_TCIA_645_1/Brats17_TCIA_645_1_flai...,LGG/Brats17_TCIA_645_1/Brats17_TCIA_645_1_seg....,LGG/Brats17_TCIA_645_1/Brats17_TCIA_645_1_t1.n...,LGG/Brats17_TCIA_645_1/Brats17_TCIA_645_1_t1ce...,LGG/Brats17_TCIA_645_1/Brats17_TCIA_645_1_t2.n...,1
282,LGG/Brats17_TCIA_650_1/Brats17_TCIA_650_1_flai...,LGG/Brats17_TCIA_650_1/Brats17_TCIA_650_1_seg....,LGG/Brats17_TCIA_650_1/Brats17_TCIA_650_1_t1.n...,LGG/Brats17_TCIA_650_1/Brats17_TCIA_650_1_t1ce...,LGG/Brats17_TCIA_650_1/Brats17_TCIA_650_1_t2.n...,1
283,LGG/Brats17_TCIA_653_1/Brats17_TCIA_653_1_flai...,LGG/Brats17_TCIA_653_1/Brats17_TCIA_653_1_seg....,LGG/Brats17_TCIA_653_1/Brats17_TCIA_653_1_t1.n...,LGG/Brats17_TCIA_653_1/Brats17_TCIA_653_1_t1ce...,LGG/Brats17_TCIA_653_1/Brats17_TCIA_653_1_t2.n...,1


#### 추출한 dictionary 저장

In [64]:
import json

In [69]:
with open("dataset/BraTS2017_filedict.json","w") as f:
    json.dump(file_dict, f, indent=4) # indent 넣어야 예쁘게 정리되서 저장됨

In [66]:
len(file_dict) # train 210개, test 75개
# pd.DataFrame(file_dict).to_csv('dataset/BraTS2017_filedict.csv')

285

### Train, Validation 나눠서 저장

In [94]:
file_dict = np.array(file_dict)

In [113]:
valid_idx = np.append(np.append(np.arange(16,20),np.arange(91,108)),np.arange(190,210))
valid_idx = np.append(np.append(valid_idx, np.arange(218,220)),np.arange(273,285))
train_idx = np.setdiff1d(np.arange(0,len(file_dict)),valid_idx)

In [115]:
valid_idx

array([ 16,  17,  18,  19,  91,  92,  93,  94,  95,  96,  97,  98,  99,
       100, 101, 102, 103, 104, 105, 106, 107, 190, 191, 192, 193, 194,
       195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207,
       208, 209, 218, 219, 273, 274, 275, 276, 277, 278, 279, 280, 281,
       282, 283, 284])

In [114]:
print(f'train length: {len(train_idx)}, validation length: {len(valid_idx)}, total: {len(file_dict)}')

train length: 230, validation length: 55, total: 285


In [121]:
final_dict = {
    'description':'BraTS2017 (train:test=8:2, class 비율 동일하게)',
    'labels':{
        '0':'HGG',
        '1':'LGG',
    },
    'numTest':57,
    'numTrain':228,
    'validation':list(file_dict[valid_idx]),'training':list(file_dict[train_idx])
    }

In [122]:
with open("dataset/BraTS2017.json","w") as f:
    json.dump(final_dict, f, indent=4) # indent 넣어야 예쁘게 정리되서 저장됨